In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [1]:
# ===== Open Screen =====
import pygame
import sys
import time
import subprocess  # 新增 import

pygame.init()
pygame.mixer.init()
pygame.font.init()

tooltip_alpha = 0
tooltip_visible = False

# 基本設置
info = pygame.display.Info()
WIDTH, HEIGHT = info.current_w, info.current_h
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Cashflow Master")
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
title_font = pygame.font.SysFont(None, 120)
info_title_font = pygame.font.SysFont(None, 80)
small_font = pygame.font.SysFont(None, 80)
font = pygame.font.SysFont(None, 50)
dialog_font = pygame.font.SysFont(None, 50)

# 顏色
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
LIGHT_BLUE = (0, 150, 255)
DARK_BLUE = (86,105,179)
YELLOW = (229,187,75)
ORANGE = (224,124,67)

# 聲音資源
click_sound = pygame.mixer.Sound("sounds/start_click.wav")
type_sound = pygame.mixer.Sound("sounds/typewriter.wav")
type_sound.set_volume(0.3) # 可依需要調整音量（0 ~ 1）
type_channel = pygame.mixer.Channel(1) # 專屬聲道分配給打字機：使用 mixer 的 Channel(1) 來播放，避免與其他聲音衝突

# 圖片資源
background_img = pygame.image.load("images/background/background.png")
return_icon = pygame.image.load("images/return.png")
return_icon = pygame.transform.scale(return_icon, (60, 60))
return_rect = return_icon.get_rect(topleft=(20, 20))

# Tooltip
tooltip_font = pygame.font.SysFont(None, 32)
def draw_tooltip(text, pos):
    surf = tooltip_font.render(text, True, WHITE)
    
    padding = 10
    tooltip_w = surf.get_width() + padding * 2
    tooltip_h = surf.get_height() + padding * 2

    bg = pygame.Surface((tooltip_w, tooltip_h), pygame.SRCALPHA)  # 支援透明
    bg.fill((0, 0, 0, 0))  # 背景完全透明

    # 繪製圓角矩形背景（黑色，透明度 200）
    pygame.draw.rect(bg, (50, 50, 50, 200), (0, 0, tooltip_w, tooltip_h), border_radius=10)

    # 把文字貼上去
    bg.blit(surf, (padding, padding))

    # 最後把整個 tooltip 畫到畫面上
    screen.blit(bg, pos)


# 按鈕類別
class Button:
    def __init__(self, rect, text, value):
        self.rect = pygame.Rect(rect)
        self.text = text
        self.value = value
        self.hovered = False

    def draw(self, surface):
        color = LIGHT_BLUE if self.hovered else BLUE
        pygame.draw.rect(surface, color, self.rect, border_radius=12)
        pygame.draw.rect(surface, BLACK, self.rect, 3, border_radius=12)
        text_surf = small_font.render(self.text, True, BLACK)
        surface.blit(text_surf, text_surf.get_rect(center=self.rect.center))

    def is_clicked(self, event):
        return self.rect.collidepoint(event.pos)

# 提示文字動畫
class SlideMessage:
    def __init__(self, text, font, center_y):
        self.text = text
        self.font = title_font
        self.center_y = center_y
        self.pos_x = -800  # 從左邊螢幕外開始
        self.final_x = WIDTH // 2  # 目標位置是畫面正中央
        self.state = "slide_in"
        self.timer = 0

    def update(self):
        if self.state == "slide_in":
            self.pos_x += 40
            if self.pos_x >= self.final_x:
                self.pos_x = self.final_x
                self.state = "stay"
        elif self.state == "stay":
            self.timer += 1
            if self.timer > 60:
                self.state = "slide_out"
        elif self.state == "slide_out":
            self.pos_x += 40
            if self.pos_x > WIDTH + 400:
                self.state = "done"

    def draw(self, surface):
        if self.state == "done":
            return

        # 渲染文字
        text_surface = self.font.render(self.text, True, BLACK)

        # 設定背景框的尺寸
        box_width = surface.get_width()
        box_height = 200

        # 正確計算框的位置：讓框「以 pos_x 為中心」
        box_x = self.pos_x - box_width // 2
        box_y = self.center_y - box_height // 2

        # 繪製半透明框
        bg_surface = pygame.Surface((box_width, box_height), pygame.SRCALPHA)
        bg_surface.fill((255, 255, 255, 200))
        surface.blit(bg_surface, (box_x, box_y))

        # 計算文字位置（放在框的正中央）
        text_rect = text_surface.get_rect(center=(self.pos_x, self.center_y))
        surface.blit(text_surface, text_rect)

# 輸入框
class TextInputBox:
    def __init__(self, x, y, w, h, font):
        self.rect = pygame.Rect(x, y, w, h)
        self.color = WHITE
        self.text = ''
        self.font = font
        self.txt_surface = font.render(self.text, True, BLACK)
        self.active = False

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            self.active = self.rect.collidepoint(event.pos)
        if event.type == pygame.KEYDOWN and self.active:
            if event.key == pygame.K_RETURN:
                return "submit"
            elif event.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                if len(self.text) < 12:  # 限制名稱長度
                    self.text += event.unicode
            self.txt_surface = self.font.render(self.text, True, BLACK)

    def draw(self, surface):
        pygame.draw.rect(surface, WHITE, self.rect, border_radius=12)
        pygame.draw.rect(surface, BLACK, self.rect, 2, border_radius=12)
        surface.blit(self.txt_surface, (self.rect.x+10, self.rect.y+20))

pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# ===== Start Screen ======

# 背景音樂載入並播放（無限循環）
pygame.mixer.music.load("sounds/backgroundmusic.wav")
pygame.mixer.music.set_volume(0.5)   # 音量（0.0 ~ 1.0）
pygame.mixer.music.play(-1)          # -1 表示無限循環播放

# 顏色定義
WHITE = (255, 255, 255)
BLUE = (0, 120, 215)
LIGHT_BLUE = (0, 150, 255)
DARK_BLUE = (86,105,179)
NIGHT_BUTTON_COLOR = (200, 200, 200)
NIGHT_BUTTON_HOVER = (100, 100, 100)

# 字型
font = pygame.font.SysFont(None, 48)

# 螢幕設定
info = pygame.display.Info()
WIDTH, HEIGHT = info.current_w, info.current_h
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Cashflow Master")

# 圖片載入
background_day = pygame.transform.scale(pygame.image.load("images/background/background_daylight.png"), (WIDTH, HEIGHT))
background_night = pygame.transform.scale(pygame.image.load("images/background/background_night.png"), (WIDTH, HEIGHT))
road_day = pygame.image.load("images/road/road_daylight.png")
road_night = pygame.image.load("images/road/road_night.png")

walk1 = pygame.image.load("images/walk/walk1.png")
walk2 = pygame.image.load("images/walk/walk2.png")
walk3 = pygame.image.load("images/walk/walk3.png")
walk4 = pygame.image.load("images/walk/walk4.png")
walk1 = pygame.transform.scale(walk1, (walk1.get_width() // 2, walk1.get_height() // 2))
walk2 = pygame.transform.scale(walk2, (walk2.get_width() // 2, walk2.get_height() // 2))
walk3 = pygame.transform.scale(walk3, (walk3.get_width() // 2, walk3.get_height() // 2))
walk4 = pygame.transform.scale(walk4, (walk4.get_width() // 2, walk4.get_height() // 2))

logo_img = pygame.image.load("images/logo.png")
nightmode_icon = pygame.image.load("images/nightmode.png")
nightmode_icon = pygame.transform.scale(nightmode_icon, (60, 60))
nightmode_rect = nightmode_icon.get_rect(topleft=(1500, 900))

class Character:
    def __init__(self, x, y):
        self.images = [walk1, walk2, walk3, walk4]
        self.index = 0
        self.timer = 0
        self.fps = 60
        self.frame_rate = 8
        self.delay = self.fps // self.frame_rate
        self.x = -200
        self.y = 300

    def update(self):
        self.timer += 1
        if self.timer >= self.delay:
            self.index = (self.index + 1) % len(self.images)
            self.timer = 0

    def draw(self, surface):
        surface.blit(self.images[self.index], (self.x, self.y))

class Road:
    def __init__(self, img, y):
        self.img = img
        self.y = y
        self.x1 = 0
        self.x2 = img.get_width()
        self.speed = 2

    def set_image(self, img):
        self.img = img
        self.x1 = 0
        self.x2 = img.get_width()

    def update(self):
        self.x1 -= self.speed
        self.x2 -= self.speed
        if self.x1 <= -self.img.get_width():
            self.x1 = self.x2 + self.img.get_width()
        if self.x2 <= -self.img.get_width():
            self.x2 = self.x1 + self.img.get_width()

    def draw(self, surface):
        surface.blit(self.img, (self.x1, self.y))
        surface.blit(self.img, (self.x2, self.y))

class Button:
    def __init__(self, rect, text):
        self.rect = rect
        self.text = text

    def draw(self, surface, is_night):
        mouse_pos = pygame.mouse.get_pos()
        if is_night:
            color = NIGHT_BUTTON_HOVER if self.rect.collidepoint(mouse_pos) else NIGHT_BUTTON_COLOR
        else:
            color = LIGHT_BLUE if self.rect.collidepoint(mouse_pos) else BLUE

        pygame.draw.rect(surface, color, self.rect, border_radius=12)
        text_surf = font.render(self.text, True, WHITE)
        text_rect = text_surf.get_rect(center=self.rect.center)
        surface.blit(text_surf, text_rect)

    def is_clicked(self, event):
        return event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos)

# 初始化元素
is_night = False
road_y = HEIGHT - road_day.get_height() + 200
character = Character(300, road_y - walk1.get_height() + 10)
road = Road(road_day, road_y)
logo_rect = logo_img.get_rect(midtop=(WIDTH // 2, -150))
button = Button(pygame.Rect(WIDTH // 2 - 100, HEIGHT // 2, 200, 60), "Start Game")

clock = pygame.time.Clock()
running = True
game_started = False

while running:
    dt = clock.tick(60)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if not game_started and button.is_clicked(event):
            click_sound.play()
            pygame.mixer.music.stop()  # 停止背景音樂
            pygame.quit()              # 關閉 Pygame
            # 執行 main_game.py，假設 python 執行環境是 python3
            subprocess.run(["python3", "main_game.py"])  
            sys.exit()                 # 退出此程式

        if event.type == pygame.MOUSEBUTTONDOWN and nightmode_rect.collidepoint(event.pos):
            click_sound.play()
            is_night = not is_night
            road.set_image(road_night if is_night else road_day)
            
    current_background = background_night if is_night else background_day

    if not game_started:
        road.update()
        character.update()
        screen.blit(current_background, (0, 0))
        road.draw(screen)
        character.draw(screen)
        screen.blit(logo_img, logo_rect)
        button.draw(screen, is_night)
        screen.blit(nightmode_icon, nightmode_rect)

        mouse_pos = pygame.mouse.get_pos()
        if nightmode_rect.collidepoint(mouse_pos):
            tooltip_visible = True
            tooltip_alpha = min(255, tooltip_alpha + 15)
        else:
            tooltip_alpha = max(0, tooltip_alpha - 15)
            if tooltip_alpha == 0:
                tooltip_visible = False

        if tooltip_alpha > 0:
            tooltip_text = font.render("Night Mode", True, (255, 255, 255))
            text_rect = tooltip_text.get_rect(center=(nightmode_rect.centerx, nightmode_rect.top - 40))

            bubble_padding = 10
            bubble_rect = pygame.Rect(
                text_rect.left - bubble_padding,
                text_rect.top - bubble_padding,
                text_rect.width + 2 * bubble_padding,
                text_rect.height + 2 * bubble_padding
            )

            tooltip_surf = pygame.Surface((bubble_rect.width, bubble_rect.height), pygame.SRCALPHA)
            pygame.draw.rect(
                tooltip_surf,
                (0, 0, 0, tooltip_alpha),
                tooltip_surf.get_rect(),
                border_radius=10
            )

            screen.blit(tooltip_surf, (bubble_rect.x, bubble_rect.y))
            text_surf_alpha = tooltip_text.copy()
            text_surf_alpha.set_alpha(tooltip_alpha)
            screen.blit(text_surf_alpha, text_rect)

    else:
        # 不需要用這段，因為按 Start Game 就會執行 main_game.py 並退出此程式
        pass

    pygame.display.flip()

pygame.quit()
sys.exit()


In [ ]:
# ===== Choose Number of Players =====

# 玩家選擇設置
player_options = [
    {"label": "Single Player", "value": 1},
    {"label": "Two Players", "value": 2},
    {"label": "Three Players", "value": 3},
    {"label": "Four Players", "value": 4},
]a
buttons = []
gap_x = screen.get_width() // 3
gap_y = screen.get_height() // 3
button_w, button_h = 500, 300
for idx, option in enumerate(player_options):
    row = idx // 2
    col = idx % 2
    rect = pygame.Rect(
        gap_x * (1 + col) - button_w // 2,
        gap_y * (1 + row) - button_h // 2,
        button_w,
        button_h
    )
    buttons.append(Button(rect, option["label"], option["value"]))

# 主程式
def run():
    selected_players = None
    message = SlideMessage("Choose Number of Players", font, HEIGHT // 2)

    while True:
        screen.blit(background_img, (0, 0))

        mouse_pos = pygame.mouse.get_pos()
        hover_return = return_rect.collidepoint(mouse_pos)
        screen.blit(return_icon, return_rect)

        for btn in buttons:
            btn.hovered = btn.rect.collidepoint(mouse_pos)
            btn.draw(screen)

        if hover_return:
            draw_tooltip("Back", (return_rect.left + 70, return_rect.bottom - 50))

        message.update()
        message.draw(screen)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                click_sound.play()
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if hover_return:
                    print("Return clicked")
                    return None
                for btn in buttons:
                    if btn.is_clicked(event):
                        click_sound.play()
                        selected_players = btn.value
                        print(f"{selected_players} players selected")
                        return selected_players

        pygame.display.update()
        clock.tick(60)

if __name__ == "__main__":
    run()


In [ ]:
# ===== Enter Player's Name =====
def run():
    input_text = ""
    message = SlideMessage("Enter Your Name", font, HEIGHT // 2 - 200)
    confirming = False

    input_box = pygame.Rect(WIDTH // 2 - 250, HEIGHT // 2, 500, 80)
    enter_button = pygame.Rect(WIDTH // 2 - 100, HEIGHT // 2 + 120, 200, 60)

    active = False
    smaller_font = pygame.font.SysFont(None, 48)
    tiny_font = pygame.font.SysFont(None, 36)

    while True:
        screen.blit(background_img, (0, 0))
        mouse_pos = pygame.mouse.get_pos()
        hover_return = return_rect.collidepoint(mouse_pos)
        screen.blit(return_icon, return_rect)

        # 提示文字最先畫
        message.update()
        message.draw(screen)

        # 輸入框上方說明文字
        player_index = 1
        label_surface = small_font.render(f"Player {player_index}'s Name", True, BLACK)
        screen.blit(label_surface, (input_box.x, input_box.y - 100))

        # 輸入框背景
        pygame.draw.rect(screen, WHITE, input_box, border_radius=10)

        # 輸入框外框
        input_hover = input_box.collidepoint(mouse_pos)
        input_color = LIGHT_BLUE if active or input_hover else BLACK
        pygame.draw.rect(screen, input_color, input_box, 3, border_radius=10)

        # 顯示輸入文字
        name_surface = small_font.render(input_text, True, BLACK)
        text_y = input_box.y + (input_box.height - name_surface.get_height()) // 2
        screen.blit(name_surface, (input_box.x + 10, text_y))

        # === 閃爍游標 ===
        if active:
            cursor_x = input_box.x + 10 + name_surface.get_width() + 2
            cursor_y = text_y
            cursor_height = name_surface.get_height()

            if (pygame.time.get_ticks() // 500) % 2 == 0:
                pygame.draw.line(screen, BLACK, (cursor_x, cursor_y), (cursor_x, cursor_y + cursor_height), 2)

        # Enter 按鈕
        pygame.draw.rect(screen, LIGHT_BLUE, enter_button, border_radius=10)
        pygame.draw.rect(screen, BLACK, enter_button, 3, border_radius=10)
        enter_text = small_font.render("Enter", True, BLACK)
        screen.blit(enter_text, enter_text.get_rect(center=enter_button.center))

        # 返回提示泡泡
        if hover_return:
            draw_tooltip("Back", (return_rect.left + 70, return_rect.bottom - 50))

        # 如果正在確認，就顯示灰色遮罩 + 確認框
        if confirming:
            confirm_lines = [
                f"You are {input_text}!",
                "Are you sure? You won’t be able to change this later!"
            ]
            line_surfaces = [smaller_font.render(line, True, BLACK) for line in confirm_lines]

            max_width = max(surface.get_width() for surface in line_surfaces)
            total_text_height = sum(surface.get_height() for surface in line_surfaces) + (len(line_surfaces) - 1) * 10

            confirm_box_width = max_width + 80
            confirm_box_height = total_text_height + 150
            confirm_box = pygame.Rect(
                WIDTH // 2 - confirm_box_width // 2,
                HEIGHT // 2 - confirm_box_height // 2,
                confirm_box_width,
                confirm_box_height
            )
            confirm_ok = pygame.Rect(WIDTH // 2 - 120, confirm_box.centery + 20, 100, 50)
            confirm_cancel = pygame.Rect(WIDTH // 2 + 20, confirm_box.centery + 20, 100, 50)

            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 160))
            screen.blit(overlay, (0, 0))

            pygame.draw.rect(screen, WHITE, confirm_box, border_radius=20)
            pygame.draw.rect(screen, BLACK, confirm_box, 4, border_radius=20)

            start_y = confirm_box.top + 40
            for i, surface in enumerate(line_surfaces):
                line_x = (WIDTH - surface.get_width()) // 2
                line_y = start_y + i * (surface.get_height() + 10)
                screen.blit(surface, (line_x, line_y))

            pygame.draw.rect(screen, LIGHT_BLUE, confirm_ok, border_radius=8)
            pygame.draw.rect(screen, BLACK, confirm_ok, 2, border_radius=8)
            ok_text = tiny_font.render("Confirm", True, BLACK)
            screen.blit(ok_text, ok_text.get_rect(center=confirm_ok.center))

            pygame.draw.rect(screen, GRAY, confirm_cancel, border_radius=8)
            pygame.draw.rect(screen, BLACK, confirm_cancel, 2, border_radius=8)
            cancel_text = tiny_font.render("Cancel", True, BLACK)
            screen.blit(cancel_text, cancel_text.get_rect(center=confirm_cancel.center))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if hover_return:
                    click_sound.play()
                    return None

                if confirming:
                    if confirm_ok.collidepoint(event.pos):
                        click_sound.play()
                        print(f"Player name confirmed: {input_text}")
                        return input_text
                    elif confirm_cancel.collidepoint(event.pos):
                        click_sound.play()
                        confirming = False
                else:
                    if input_box.collidepoint(event.pos):
                        active = True
                    else:
                        active = False

                    if enter_button.collidepoint(event.pos):
                        click_sound.play()
                        confirming = True

            elif event.type == pygame.KEYDOWN:
                if not confirming:
                    if active:
                        if event.key == pygame.K_RETURN:
                            confirming = True
                            click_sound.play()
                        elif event.key == pygame.K_BACKSPACE:
                            input_text = input_text[:-1]
                        else:
                            if len(input_text) < 20:
                                input_text += event.unicode
                elif confirming: # 當確認視窗開啟時，按 Enter 鍵就等於點擊 Confirm
                    if event.key == pygame.K_RETURN:
                        click_sound.play()
                        print(f"Player name confirmed: {input_text}")
                        return input_text

        pygame.display.update()
        clock.tick(60)

if __name__ == "__main__":
    run()


In [2]:
# =====Career Assignment=====

job_to_image = {
    "Youtuber": "images/career/youtuber.png",
    "social media editor": "images/career/socialmediaeditor.png",
    "engineer": "images/career/engineer.png",
    "salesman": "images/career/salesman.png",
    
}


import random

### ------------------ 區塊二：玩家與職業管理 ------------------
class Job:
    def __init__(self, title, salary, expense, bonus_type, bonus_params):
        self.title = title
        self.salary = salary          # 每季薪資 = 月薪 × 3
        self.expense = expense        # 每季支出 = 月支出 × 3
        self.bonus_type = bonus_type  # 'annual','performance','sales','project'
        self.bonus_params = bonus_params  # dict 內含獎金細節

class Player:
    def __init__(self, name, job, initial_cash):
        self.name = name
        self.job = job
        self.cash = initial_cash
        self.base_expense = job.expense
        self.expense = job.expense
        self.assets = {}
        self.passive = {}
        self.liabilities = {}
        self.married = False
        self.children = 0
        self.divorced = False

    def net_worth(self):
        total_assets = self.cash + sum(self.assets.values())
        total_debt = sum(d['principal'] for d in self.liabilities.values())
        return total_assets - total_debt

    def update_passive_income(self):
        return sum(self.passive.values())

    def apply_inflation(self):
        self.expense = int(self.expense * 1.02)

    def can_repay(self):
        due = sum(d['repay'] for d in self.liabilities.values())
        return self.cash >= (self.base_expense + due)

    def summary(self):
        print(f"\n\U0001F4CA【{self.name} - {self.job.title}】")
        print(f"  現金：{self.cash}  支出(季)：{self.expense}")
        print(f"  被動收入(季)：{self.update_passive_income()}  資產：{self.assets}")
        debt_str = {k: v['principal'] for k, v in self.liabilities.items()}
        print(f"  債務：{debt_str}  淨資產：{self.net_worth()}")

def create_all_roles():
    return [
        ("A", Job("Youtuber", 0, 60000*3, 'project', {'min':40000,'max':60000}), 200000),
        ("B", Job("社群小編", 40000*3, 20000*3, 'performance', {'min':5000,'max':15000,'rate':0.7}), 80000),
        ("C", Job("工程師", 85000*3, 30000*3, 'annual', {'min':50000,'max':150000}), 120000),
        ("D", Job("業務", 35000*3, 28000*3, 'sales', {'min':10000,'max':50000,'rate':0.6}), 150000),
    ]

players = [
    Player("Mike", Job("Youtuber", 0, 60000*3, 'project', {'min':40000,'max':60000}), 200000),
    Player("Taylor", Job("social media editor", 40000*3, 20000*3, 'performance', {'min':5000,'max':15000,'rate':0.7}), 80000),
    Player("Betty", Job("engineer", 85000*3, 30000*3, 'annual', {'min':50000,'max':150000}), 120000),
    Player("Charlie", Job("salesman", 35000*3, 28000*3, 'sales', {'min':10000,'max':50000,'rate':0.6}), 150000),
]

def generate_dialog(player):
    return f"I'm a {player.job.title}. I'm ready for it!"

def draw_layout(player):
    screen.blit(background_img, (0, 0))

    # 左邊資訊欄
    info_rect = pygame.Rect(40, 40, 600, 950)
    pygame.draw.rect(screen, YELLOW, info_rect, border_radius=10)
    pygame.draw.rect(screen, ORANGE, info_rect, 5, border_radius=10)

    # 左邊資訊標題
    title_surf = info_title_font.render("Player's Info", True, ORANGE)
    screen.blit(title_surf, (info_rect.x + 30, info_rect.y + 35))
    
    # 左邊資訊文字
    labels = ["Name", "Job", "Cash", "Expense (qtr)", "Passive Income (qtr)", "Assets", "Debts", "Net Assets"]
    values = [
        f"{player.name}",
        f"{player.job.title}",
        f"${player.cash}",
        f"${player.expense}",
        f"${player.update_passive_income()}",
        f"${sum(player.assets.values()) if player.assets else 0}",
        f"${sum(d['principal'] for d in player.liabilities.values()) if player.liabilities else 0}",
        f"${player.net_worth()}"
    ]
    for i, (label, value) in enumerate(zip(labels, values)):
        label_surf = font.render(label, True, BLACK)
        value_surf = font.render(value, True, BLACK)
        y_offset = info_rect.y + 120 + i * 80
        screen.blit(label_surf, (info_rect.x + 30, y_offset))
        screen.blit(value_surf, (info_rect.right - value_surf.get_width() - 30, y_offset))

    # 右上角角色圖
    job_img = pygame.image.load(job_to_image.get(player.job.title, "images/career/youtuber.png")) # 載入圖片
    original_width, original_height = job_img.get_size() # 取得原始圖片尺寸
    new_width = 600
    aspect_ratio = original_height / original_width # 根據原比例計算新高度
    new_height = int(new_width * aspect_ratio)
    job_img = pygame.transform.scale(job_img, (new_width, new_height)) # 縮放圖片
    screen.blit(job_img, (WIDTH - new_width - 250, 50))  # 減去 new_width 是為了右側貼齊

    # 對話框
    dialog_box = pygame.Rect(680, 750, 980, 240)
    pygame.draw.rect(screen, WHITE, dialog_box, border_radius=10)
    pygame.draw.rect(screen, GRAY, dialog_box, 5, border_radius=10)

    # 名字框
    name_surface = info_title_font.render(player.name, True, BLACK)
    padding_x = 30
    padding_y = 20
    name_box_width = name_surface.get_width() + padding_x * 2
    name_box_height = name_surface.get_height() + padding_y * 2
    name_box = pygame.Rect(680, 700, name_box_width, name_box_height)
    pygame.draw.rect(screen, LIGHT_BLUE, name_box, border_radius=10)
    pygame.draw.rect(screen, DARK_BLUE, name_box, 5, border_radius=10)
    screen.blit(name_surface, (name_box.x + padding_x, name_box.y + padding_y))

def draw_typing_text(text, x, y, font, surface, player, color=BLACK, speed=30):
    displayed_text = ""
    type_channel.play(type_sound, loops=-1)
    for char in text:
        displayed_text += char
        draw_layout(player)
        text_surface = font.render(displayed_text, True, color)
        surface.blit(text_surface, (x, y))
        pygame.display.update()
        pygame.time.delay(speed)
    type_channel.stop()

def wait_for_next_or_timeout(timeout_sec):
    start_time = time.time()
    while True:
        elapsed = time.time() - start_time
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                return
        if elapsed >= timeout_sec:
            return
        clock.tick(60)

def main():
    for player in players:
        dialog = generate_dialog(player)
        draw_typing_text(dialog, 711, 820, dialog_font, screen, player)
        wait_for_next_or_timeout(5)
    pygame.quit()

if __name__ == '__main__':
    main()
